In [1]:
import os
from dotenv import load_dotenv
import nibabel as nib
from nibabel.orientations import aff2axcodes
import h5py
import pandas as pd
import numpy as np
from tqdm import tqdm

from utils.registration import nifit_info, correct4registration, slice2chunk_align
from utils.preproc import FUSCleaner

from utils.glm import run_glm_analysis
from nilearn.glm.contrasts import compute_contrast
from scipy.ndimage import affine_transform
import SimpleITK as sitk

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from joblib import Parallel, delayed

In [2]:
load_dotenv()
base_dir = os.getenv('BASE_DIR')
task_dir = os.path.join(base_dir, 'task_data')
data_dir = os.path.join(task_dir, 'audiocue_acrossday', 'grouper', '1002')
register_dir = os.path.join(base_dir, 'slice2chunk_grouper', '100224_5-5')
figure_dir = os.path.join(base_dir, 'figures')

os.makedirs(figure_dir, exist_ok=True)

In [3]:
raw_file = os.path.join(data_dir, 'Grouper_1002_audiocue_plane5.5_2Dscan_1.nii')
raw_file2 = os.path.join(data_dir, 'Grouper_1002_audiocue_plane5.5_2Dscan_1.nii.gz')

# correct 2D image for landmark registration
# outfile = os.path.join(data_dir, 'Grouper_1002_audiocue_corrected.nii.gz')
# correct4registration(raw_file2, outfile)
nifit_info(raw_file), nifit_info(raw_file2)

pixdim[1,2,3] should be positive; setting to abs of pixdim values


(((128, 1, 153, 9000),
  array([[-0.11      ,  0.        ,  0.        ,  6.98499996],
         [ 0.        ,  0.40000001,  0.        , -0.        ],
         [ 0.        ,  0.        ,  0.09856   , -7.49055985],
         [ 0.        ,  0.        ,  0.        ,  1.        ]]),
  ('L', 'A', 'S')),
 ((128, 1, 153),
  array([[ 0.11      ,  0.        ,  0.        , -0.        ],
         [ 0.        ,  0.40000001,  0.        , -0.        ],
         [ 0.        ,  0.        , -0.09856   ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  1.        ]]),
  ('R', 'A', 'I')))

In [6]:
import nibabel as nib
import numpy as np
from nibabel.orientations import aff2axcodes

def transform_and_correct(input_file, output_file):
    img = nib.load(input_file)
    data = img.get_fdata()
    affine = img.affine

    # Flip the x-axis (L-R flip)
    data = np.flip(data, axis=0)

    # Adjust the affine matrix
    new_affine = affine.copy()
    new_affine[0, 0] = -new_affine[0, 0]  # Flip x-axis
    new_affine[0, 3] = 0  # Reset x translation to 0
    new_affine[1, 3] = 0  # Reset y translation to 0
    new_affine[2, 3] = 0  # Reset z translation to 0

    # Swap x and y in the affine matrix
    new_affine[[0, 1]] = new_affine[[1, 0]]

    # Ensure z-axis is positive (for 'I' orientation)
    if new_affine[2, 2] < 0:
        new_affine[2, 2] = -new_affine[2, 2]
        data = np.flip(data, axis=2)

    print("Final Affine:\n", new_affine)
    print("Orientation:", aff2axcodes(new_affine))

    # Create the final image
    new_img = nib.Nifti1Image(data, new_affine, img.header)

    # Save the transformed and corrected image
    nib.save(new_img, output_file)

    print(f"Transformed and corrected image saved to: {output_file}")

In [7]:

# Usage
input_file = raw_file  # Use the path to your input file
output_file = os.path.join(data_dir, 'Grouper_1002_audiocue_transformed_corrected.nii.gz')
transform_and_correct(input_file, output_file)

# Verify the transformation
print("\nOriginal image info:")
print(nifit_info(input_file))
print("\nTransformed and corrected image info:")
print(nifit_info(output_file))

pixdim[1,2,3] should be positive; setting to abs of pixdim values


Final Affine:
 [[0.         0.40000001 0.         0.        ]
 [0.11       0.         0.         0.        ]
 [0.         0.         0.09856    0.        ]
 [0.         0.         0.         1.        ]]
Orientation: ('A', 'R', 'S')


pixdim[1,2,3] should be positive; setting to abs of pixdim values


Transformed and corrected image saved to: /Users/yibeichen/Desktop/fusi/task_data/audiocue_acrossday/grouper/1002/Grouper_1002_audiocue_transformed_corrected.nii.gz

Original image info:
((128, 1, 153, 9000), array([[-0.11      ,  0.        ,  0.        ,  6.98499996],
       [ 0.        ,  0.40000001,  0.        , -0.        ],
       [ 0.        ,  0.        ,  0.09856   , -7.49055985],
       [ 0.        ,  0.        ,  0.        ,  1.        ]]), ('L', 'A', 'S'))

Transformed and corrected image info:
((128, 1, 153, 9000), array([[0.        , 0.40000001, 0.        , 0.        ],
       [0.11      , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.09856   , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]]), ('A', 'R', 'S'))
